<a href="https://colab.research.google.com/github/PedroOlavo/ArtigosMedium/blob/main/Energia_em_Minas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np

Leitura dos dados

In [2]:
dados = pd.read_excel('/content/drive/MyDrive/BD SIGA 01012021.xlsx', header=1)
dados.head()

,Empreendimento,CEG,UF,Fonte,Fase,Origem,Tipo,Tipo de Atuação,Combustível Final,Entrada em Operação,Potência Outorgada (kW),Potência Fiscalizada (kW),Garantia Fisica (kW),Geração Qualificada,Latitude (GMS),Longitude (GMS),Início Vigência,Fim Vigência,Proprietário / Regime de Exploração,Cod e Desc Sub-Bacia,Município (s)
0,14 de Julho,UHE.PH.RS.000012-4,RS,UHE,Operação,Hídrica,Potencial hidráulico,Concessão,Potencial hidráulico,25/12/2008,100000.0,100710.0,47500,Não,"29°3'53,136""S","51°40'29,824""W",2001-03-15,2036-03-15,100% para COMPANHIA ENERGÉTICA RIO DAS ANTAS (...,86 - Taquari,"Bento Gonçalves - RS, Cotiporã - RS"
1,29 de Outubro,UTE.PE.SC.029920-0,SC,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,07/10/2008,62.0,62.0,0,Não,"26° 29' 9,370"" S","49° 5' 10,100"" W",NaT,NaT,100% para Condomínio Residencial 29 de Outubro...,NaN,Jaraguá do Sul - SC
2,300KVA SCANIA,UTE.PE.MG.032686-0,MG,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,01/06/2013,240.0,240.0,0,Não,"21°46'53,08""S","46°36'22,97""W",NaT,NaT,100% para Mineração Curimbaba Ltda. (REG),NaN,Poços de Caldas - MG
3,3M Itapetininga,UTE.PE.SP.028642-7,SP,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,12/06/2002,3840.0,3840.0,0,Não,"23°36'3,19""S","48°5'41,69""W",NaT,NaT,100% para 3M do Brasil Ltda (REG),NaN,Itapetininga - SP
4,3M Sumaré,UTE.PE.SP.028919-1,SP,UTE,Operação,Fóssil,Petróleo,Autorização,Óleo Diesel,01/10/2001,6400.0,6400.0,0,Não,"22°49'15,73""S","47°11'24,3""W",2003-06-20,2033-06-20,100% para 3M do Brasil Ltda (APE),NaN,Sumaré - SP


Verificando o nome das colunas da planilha

In [3]:
dados.columns

Index(['Empreendimento', 'CEG', 'UF', 'Fonte', 'Fase', 'Origem', 'Tipo',
       'Tipo de Atuação', 'Combustível Final', 'Entrada em Operação',
       'Potência Outorgada (kW)', 'Potência Fiscalizada (kW)',
       'Garantia Fisica (kW)', 'Geração Qualificada', 'Latitude (GMS)',
       'Longitude (GMS)', 'Início Vigência', 'Fim Vigência',
       'Proprietário / Regime de Exploração', 'Cod e Desc Sub-Bacia',
       'Município (s)'],
      dtype='object')

Analisando como os estdos estão listados

In [4]:
dados.UF.unique()

array(['RS', 'SC', 'MG', 'SP', 'AC', 'RO', 'GO', 'PA', 'BA', 'RN', 'PR',
       'MT', 'RJ', 'AL', 'TO', 'AM', 'PI', 'ES', 'RR', 'DF', 'MS', 'PE',
       'CE', 'PB', 'MA', 'AP', 'SE'], dtype=object)

Selecionando apenas as observações relativas à Minas Gerais

In [5]:
minas = dados.loc[dados['UF']=='MG']
minas.head()

,Empreendimento,CEG,UF,Fonte,Fase,Origem,Tipo,Tipo de Atuação,Combustível Final,Entrada em Operação,Potência Outorgada (kW),Potência Fiscalizada (kW),Garantia Fisica (kW),Geração Qualificada,Latitude (GMS),Longitude (GMS),Início Vigência,Fim Vigência,Proprietário / Regime de Exploração,Cod e Desc Sub-Bacia,Município (s)
2,300KVA SCANIA,UTE.PE.MG.032686-0,MG,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,01/06/2013,240.0,240.0,0,Não,"21°46'53,08""S","46°36'22,97""W",NaT,NaT,100% para Mineração Curimbaba Ltda. (REG),NaN,Poços de Caldas - MG
15,ABC,UTE.PE.MG.030905-2,MG,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,01/12/2011,184.0,184.0,0,Não,"20° 44' 59,316"" S","44° 44' 9,528"" W",NaT,NaT,100% para Adição Distribuição Express Ltda. (REG),NaN,Oliveira - MG
16,ABC,UTE.PE.MG.031170-7,MG,UTE,Operação,Fóssil,Petróleo,Registro,Óleo Diesel,01/08/2011,400.0,400.0,0,Não,"20° 7' 25,392"" S","44° 55' 50,484"" W",NaT,NaT,100% para Adição Distribuição Express Ltda. (REG),NaN,Divinópolis - MG
30,AC III,UFV.RS.MG.037403-2,MG,UFV,Construção não iniciada,Solar,Radiação solar,Autorização,Radiação solar,-,34000.0,0.0,0,Não,"15° 21' 32,690"" S","43° 48' 25,890"" W",2019-08-09,2054-08-09,100% para VALE S.A. (PIE),NaN,Jaíba - MG
31,AC IV,UFV.RS.MG.037404-0,MG,UFV,Construção não iniciada,Solar,Radiação solar,Autorização,Radiação solar,-,40000.0,0.0,0,Não,"15° 21' 54,430"" S","43° 47' 40,380"" W",2019-08-09,2054-08-09,100% para VALE S.A. (PIE),NaN,Jaíba - MG


Importação das bibliotecas para gerar os gráficos

In [6]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

Criação de uma nova tabela para contar quantas usinas estão em operação, construção ou não iniciadas

In [7]:
fase = minas.Fase.unique()
n_fase = minas.Fase.value_counts()

n_u_fase = {'Fase': fase, 'Número de usinas': n_fase}
n_u_fase = pd.DataFrame(n_u_fase)
n_u_fase

,Fase,Número de usinas
Operação,Operação,753
Construção não iniciada,Construção não iniciada,118
Construção,Construção,6


Gerando o gráfico para visualizar a quantidade de usinas em cada fase

In [8]:
bar_plot1 = px.bar(n_u_fase, x='Fase', y='Número de usinas', color='Fase', text='Número de usinas')

bar_plot1.update_layout(title_text='Quantidade de usinas em operação, em construção ou em espera em Minas',
                        yaxis_title = 'Número de unidades geradoras')
                        
bar_plot1.update_traces(texttemplate='%{text:}', textposition='outside')
bar_plot1.show()


Selecionando apenas as usinas que já estão em operação

In [33]:
usinas_op = minas.loc[minas['Fase']=='Operação']
usinas_op.rename(columns={'Potência Outorgada (kW)': 'Potência_kW'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Criando uma tabela para saber quantas unidades geradoras de cada tipo há no estado

In [34]:
origem = minas.Origem.unique()
n_origem = minas.Origem.value_counts()

origens = {'Origem': origem, 'Número de Usinas': n_origem}
origens = pd.DataFrame(origens)
origens

,Origem,Número de Usinas
Fóssil,Fóssil,365
Hídrica,Solar,299
Solar,Hídrica,132
Biomassa,Biomassa,80
Eólica,Eólica,1


Criação de um gráfico para mostrar o número de usinas de cada tipo

In [36]:
bar_plot2 = px.bar(origens, x='Origem', y='Número de Usinas', color='Origem', text='Número de Usinas')

bar_plot2.update_layout(title_text='Número de usinas para cada tipo de origem da energia elétrica produdiza em Minas Gerais',
                        yaxis_title = 'Número de unidades geradoras')
                        
bar_plot2.update_traces(texttemplate='%{text:}', textposition='outside')
bar_plot2.show()

Criação de uma tabela para mostrar quanto de potência cada tipo de geração produz no estado

In [37]:
capacidade = []
for i in origem:
  soma = usinas_op.loc[usinas_op['Origem']==i].Potência_kW.sum()
  soma = (soma / 1000).round(2)
  capacidade.append(soma)

cap_por_origem = {'Origem': origem, 'Potência produzida (MW)': capacidade}
cap_por_origem = pd.DataFrame(cap_por_origem)
cap_por_origem


,Origem,Potência produzida (MW)
0,Fóssil,947.97
1,Solar,541.82
2,Hídrica,13528.75
3,Biomassa,1707.72
4,Eólica,0.16


In [ ]:
Criação do gráfico de capacidade geradora de cada tipo de usina, em MW

In [38]:
bar_plot3 = px.bar(cap_por_origem, x='Origem', y='Potência produzida (MW)', color='Origem', text='Potência produzida (MW)')

bar_plot3.update_layout(title_text='Capacidade de geração para cada tipo de origem da eletricidade produzida em Minas Gerais',
                        yaxis_title = 'Megawatts gerados')
                        
bar_plot3.update_traces(texttemplate='%{text:}', textposition='outside')
bar_plot3.show()